<center style="font-size:48px;">Master Notebook</center>
<br>

This is a quick summary of the answers to the propes questions. For important aspects to this analysis please viist the notebooks that accompan this one. 

The reprository can be found at [https://github.com/cvorsanger/COMP4449Final](https://github.com/cvorsanger/COMP4449Final) 

In [ ]:
import pandas as pd

buys = pd.read_csv('../CheckPoints/Buys_Clean.csv', parse_dates=['timestamp'], index_col=0)

def findMax(x):
    # Returns the the place with the maximum number of purchases
    return x.groupby('location', as_index =False).count().sort_values('timestamp', ascending =False).head(1)['location'].values

## Question 1

Describe common daily routines for GAStech employees. What does a day in the life of a typical GAStech employee look like?

For average employee at GASTech their day consists of waking up and then going to get coffee before getting to te office at around 7AM. Then they are either in office or running errands; typically Facilities; until noon. At noon until 3PM is lunch time were we see people going to resturants. Then back to the office. Around 5PM we see people start to leave the office for resturants or their home. By 9PM most people are home.

We can see this by looking the most popular buys per hour.

In [ ]:
# Find the places
popular = {}
for hour in buys.hour.unique():
    x = findMax(buys[buys.hour == hour])
    popular[hour] =x 
# Display
popular = pd.DataFrame(popular).transpose().sort_index()
popular.columns = ['Place']
popular

And a timelapse heatmap

In [ ]:
from IPython.display import IFrame

IFrame(src='../images/Clean/Timelapse.html', width=700, height=600)


During the weekend we see people go to the golf course anfd the musuem to have fun. Much of the same resturants and coffee places are visited on the weekends.  One key difference is that nobody goes to the office on weekends.

## Question 2

Identify up to twelve unusual events or patterns that you see in the data.

There were 12 unusual events identified in this dataset; although there are probably much more. The behaviors identified were:

1. Isia Vann - Morning of the the 7th
1. Sten Sanjorge Jr.- High Purchase on the 18th
1. Claudio Hawelon - Went to the airport on the 10th
1. Lucas Alcazar - High Purchase on the 13th
1. Ingrid	Barranco - Multiple Visits to Hippokampos on the 11th
1. Hideki Cocinaro - Mutiple Purchases on the 11th
1. Felix Balas - At Frank's Fuel on the 18th
1. Lucas Alcazar - Odd Purchase on the 13th
1. Nils Calixto - Odd Purchase on the 7th 
1. Varro Awelon - Missed Daily Behavior in the 10th
1. Lars Azada - Missed Dailey Coffee Stop on the 15th
1. Willem Vasco-Pais - Odd Time of Purchase on the 10th

For a more indepth analysis of each of the twelve events please visit the [Question 2 notebook](./ObservationsQ2.ipynb)

## Question 3


Like most datasets, the data you were provided is imperfect, with possible issues such as missing data, conflicting data, data of varying resolutions, outliers, or other kinds of confusing data. Considering data is primarily spatiotemporal, describe how you identified and addressed the uncertainties and conflicts inherent in this data to reach your conclusions in questions 1 and 2.

This dataset proved to be especially dirty. There needed to be thought on how to merge the data, handle inconsistencies in both purchase times and location times, and how to handle outliers. Without careful consideration of these factors our analysis could not be trusted.

For the merged locations dataframe. I had to consider a few things. One such thing was the varying time deltas between data points. I was able to find this by inspecting the time-lapse heatmaps. By looking at the spread and color intensity it was easy to see that there are a varying amount of data points per hour. Digging deeper  and looking at a single individual confirms that sometimes the location data was reported every second, sometimes once a minute, and sometimes there is nothing for hours. Reconciliation of this aspect was done through resampling the data. For each person i resampled their location to one minute intervals (the file size was too big for GitHub if I did a one second sample). This allowed the time deltas to be more consistent at one minute. However, there were still "holes" sporadically during the day and at night. An easy solution could be to backfill null values with the previous location. However, this would greatly grow the dataset without adding any valuable information and potentially skewing the data. I therefore decided to do nothing about these holes.

The resampling also helped for the situation where two location records for a person at a single timestamp. Resampling would automatically combining these into one data point.

Another imperfection I came across was the fact that the truck drivers had to share vehicles. This was due to the fact that they didn’t have a company car they were allowed to use for personal use.

However, we were given information about the five different trucks that were shared. We could assume that these were being driven by someone with a Truck Driver job title. If someone other then a Truck Driver were to drive one of these vehicles, it would be suspicious and would show up graphical on one of our heatmaps. It was possible to investigate purchase data in order to ascertain who was the driver. 

Lastly for the location data I had to consider what to do with the outliers. Using a violin plot I see that there were some gps points considered outliers. However, they were not so extreme to hint that there was something off with the readings. Because the point of the project to identify outliers in order to make recommendations, the outliers contained valuable information that shouldn't necessarily be altered or dropped. For this reason, I decided to keep any potential outliers that way I could inspect them and determine if they were real and relevant to the investigation.

The purchases data wasn’t without its headaches as well. As I previously mentioned I found during EDA that there were many of the same purchases detailed in the loyalty card and the credit card files (about 70% of the transactions). I was able to detect this by sampling the data for a single individual and comparing the two sties of purchases. I did find two differences between the duplicate purchases. I also noticed that sometimes the dollar amount of the purchases were different with the credit card prices seemingly much more prone to outliers. Additionally, the loyalty data just gave you the day while the credit card dataset also gave you the time of the purchase in certain situations.

To solve this I did a custom merge. I created a finction that went through each transaction and identified the duplicate purchases by looking at name, date, and cents amount of the price. I then kept the credit card timestamp as it was more detailed and the loyalty card price; more on why in the next bullet point). I kept track of all of the loyalty purchases through a boolean variable that was true if the purchase was detailed in the loyalty file.

As mentioned I took the loyalty card purchase price when combining the two files. This was due to outliers in the pricing. I was able to identify these through violin plots and in the inspection mentioned in the previous section. An interesting fact and the reason why I choose to hold onto the loyalty price, was that the credit card dataset tended to be much more prone to these outliers. The outliers were trickier to deal with as the whole purpose of this analysis was outlier detection. Deciphering between actually outliers (suspects) and bad data was nearly impossible. I therefore, elected to keep them as they might contain valuable information relevant to the investigation.

Another interesting observation I found was the time that purchase transactions were dated. As mentioned, the loyalty card file only gave the date of the purchase with no time. The credit card file also gave time however, it seemed that only some transactions were reported in real time. In other instances the transactions were updated in bulk, typically at the end of the day. Additionally, looking at a sample of some the purchases at an individual place, sometimes they reported transactions in bulk  sometimes they were reported in real time, and there didn't seem to be a pattern. I had to keep these two factors in mind when analyzing the purchase data. There wasn’t much I could do about them, so when looking at the time I had to have some doubt about the time.

Lastly, there were more employees with purchases then there were assigned a car. I found this by looking for null values after combing the loyalty car, credit card, and car assignment files. These employees were among a handful that didn’t car assigned to them and didn’t work as a truck diver. Unfortunately, I wasn't able to get the job information for these employees since this information was all in the car assignments file. These were still valuable data points so I couldn't drop them. To handle this I filled these null values with the value "Other”. This allowed me to still analyze these points and to encode the job information before utilizing machine learning.ential outliers that way I could inspet them and deteremine if they were real and relevant to the investigation.